In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data/raw/beer_reviews.csv')
df

brewery_id                 brewery_name  review_time  review_overall  \
0             10325              Vecchio Birraio   1234817823             1.5   
1             10325              Vecchio Birraio   1235915097             3.0   
2             10325              Vecchio Birraio   1235916604             3.0   
3             10325              Vecchio Birraio   1234725145             3.0   
4              1075      Caldera Brewing Company   1293735206             4.0   
...             ...                          ...          ...             ...   
1586609       14359  The Defiant Brewing Company   1162684892             5.0   
1586610       14359  The Defiant Brewing Company   1161048566             4.0   
1586611       14359  The Defiant Brewing Company   1160702513             4.5   
1586612       14359  The Defiant Brewing Company   1160023044             4.0   
1586613       14359  The Defiant Brewing Company   1160005319             5.0   

         review_aroma  review_appearance review_profilename  \
0                 2.0                2.5            stcules   
1                 2.5                3.0            stcules   
2                 2.5                3.0            stcules   
3                 3.0                3.5            stcules   
4                 4.5                4.0     johnmichaelsen   
...               ...                ...                ...   
1586609           4.0                3.5         maddogruss   
1586610           5.0                2.5          yelterdow   
1586611           3.5                3.0           TongoRad   
1586612           4.5                4.5           dherling   
1586613           4.5                4.5               cbl2   

                             beer_style  review_palate  review_taste  \
0                            Hefeweizen            1.5           1.5   
1                    English Strong Ale            3.0           3.0   
2                Foreign / Export Stout            3.0           3.0   
3                       German Pilsener            2.5           3.0   
4        American Double / Imperial IPA            4.0           4.5   
...                                 ...            ...           ...   
1586609                     Pumpkin Ale            4.0           4.0   
1586610                     Pumpkin Ale            2.0           4.0   
1586611                     Pumpkin Ale            3.5           4.0   
1586612                     Pumpkin Ale            4.5           4.5   
1586613                     Pumpkin Ale            4.5           4.5   

                      beer_name  beer_abv  beer_beerid  
0                  Sausa Weizen       5.0        47986  
1                      Red Moon       6.2        48213  
2        Black Horse Black Beer       6.5        48215  
3                    Sausa Pils       5.0        47969  
4                 Cauldron DIPA       7.7        64883  
...                         ...       ...          ...  
1586609      The Horseman's Ale       5.2        33061  
1586610      The Horseman's Ale       5.2        33061  
1586611      The Horseman's Ale       5.2        33061  
1586612      The Horseman's Ale       5.2        33061  
1586613      The Horseman's Ale       5.2        33061  

[1586614 rows x 13 columns]

In [3]:
# Select relevant features
features = ['brewery_name', 'review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'beer_abv']
target = 'beer_style'

X = df[features]
y = df[target]

In [4]:
# Convert categorical variables into numerical ones
le = LabelEncoder()
X['brewery_name'] = le.fit_transform(X['brewery_name'])


/tmp/ipykernel_145/4279998631.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['brewery_name'] = le.fit_transform(X['brewery_name'])


In [5]:
# Handle missing values
X = X.fillna(X.mean())


In [6]:
# Normalize the numerical variables
sc = StandardScaler()
X = sc.fit_transform(X)


In [7]:
# Convert the target variable into numerical
y = le.fit_transform(y)


In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Model Building and Training:

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# Build the neural network model
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(set(y)), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Use early stopping to exit training if validation loss is not decreasing after certain epochs (patience)
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, callbacks=[earlystopping])


Epoch 1/100
19833/19833 [==============================] - 116s 6ms/step - loss: 3.2154 - accuracy: 0.1995 - val_loss: 3.0914 - val_accuracy: 0.2161
Epoch 2/100
19833/19833 [==============================] - 110s 6ms/step - loss: 3.0260 - accuracy: 0.2330 - val_loss: 2.9726 - val_accuracy: 0.2457
Epoch 3/100
19833/19833 [==============================] - 111s 6ms/step - loss: 2.9348 - accuracy: 0.2532 - val_loss: 2.9020 - val_accuracy: 0.2594
Epoch 4/100
19833/19833 [==============================] - 113s 6ms/step - loss: 2.8689 - accuracy: 0.2681 - val_loss: 2.8407 - val_accuracy: 0.2749
Epoch 5/100
19833/19833 [==============================] - 112s 6ms/step - loss: 2.8162 - accuracy: 0.2807 - val_loss: 2.8041 - val_accuracy: 0.2872
Epoch 6/100
19833/19833 [==============================] - 113s 6ms/step - loss: 2.7730 - accuracy: 0.2919 - val_loss: 2.7566 - val_accuracy: 0.2969
Epoch 7/100
19833/19833 [==============================] - 116s 6ms/step - loss: 2.7358 - accuracy: 0.3020

Model Evaluation:

In [13]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Make predictions
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Print the classification report
print(classification_report(y_test, y_pred))

# Print the confusion matrix
print(confusion_matrix(y_test, y_pred))


9917/9917 [==============================] - 23s 2ms/step


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.40      0.09      0.15      1568
           1       0.41      0.67      0.51      6121
           2       0.24      0.26      0.25      9143
           3       0.59      0.38      0.46      1912
           4       0.46      0.53      0.49      5336
           5       0.64      0.25      0.36      2212
           6       0.20      0.04      0.07      2517
           7       0.48      0.45      0.46      5022
           8       0.00      0.00      0.00       318
           9       0.47      0.64      0.54     17196
          10       0.40      0.29      0.34      1146
          11       0.54      0.51      0.52     10104
          12       0.41      0.80      0.54     23406
          13       0.47      0.35      0.40       855
          14       0.28      0.42      0.34     12647
          15       0.28      0.11      0.16      1847
          16       0.29      0.45      0.36      4900
          17       0.34    

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
# Model evaluation
_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy*100}')


Test Accuracy: 41.29073619842529
